# surrealdb

testing [surrealdb.rs](https://crates.io/crates/surrealdb) rust client using [jupyter labs notebooks](https://jupyter.org/try)

In [2]:
// install packages
:dep color-eyre = "0.6.2"
// :dep uuid = { version = "1.3.0", features = ["v4", "macro-diagnostics", "fast-rng", "serde"] }
:dep serde = "1.0.189"
:dep serde_json = "1.0.107"
:dep tokio = { version = "1.33.0", features = ["full"] }
:dep surrealdb = "1"
:dep ulid = "1.1.0"
:dep nanoid = "0.4.0"
:dep chrono = "0.4.31"

# surrealdb-cli

the following command will start a surreal sql command-line prompt to query the database

```sh
# run the surreal db sql cli
surreal sql --conn http://localhost:8000 --user root --pass root --ns labs --db labs --pretty --
json
```

surreal db has various types of unique identifiers. Under the hood it seems to be using the `nanoid` crate to generate new IDs but it also has support for both `ulid` and `uuid`.

```sh
# create person with uuid
❯ CREATE person:uuid();
[
    {
        "id": "person:⟨018b6377-0003-7307-a07d-c351272be86b⟩"
    }
]

# create person with ulid
❯ CREATE person:ulid();
[
        
    {
        "id": "person:01HDHQD6X9NKMTSSM0NZKK47WR"
    }
]
```

In [9]:
use color_eyre::eyre::Result;
use serde::{Deserialize, Serialize};
use serde_json::json;
use std::borrow::Cow;
use surrealdb::sql;
use surrealdb::sql::Thing;
use surrealdb::Surreal;
use surrealdb::engine::remote::ws::Ws;
use surrealdb::opt::auth::Root;
use chrono::{DateTime, Utc};


In [10]:

#[derive(Debug, Serialize, Deserialize)]
struct Name {
    firstname: String,
    lastname: String,
}

#[derive(Debug, Serialize, Deserialize)]
pub struct Identity {
    email: String,
    login: String,
    password: String,
    preferred: bool,
    providers: Vec<Provider>,
}

#[derive(Debug, Serialize, Deserialize)]
pub struct Provider {
    name: String,
}

#[derive(Debug, Serialize, Deserialize)]
struct Person {
    #[serde(skip_serializing)]
    id: Option<Thing>,
    nickname: String,
    name: Name,
    identity: Identity,
    created: DateTime<Utc>, // UTC timestamp for creation time
    updated: DateTime<Utc>, // UTC timestamp for update time
}

#[derive(Debug, Serialize, Deserialize)]
struct NewPerson {
    name: Name,
    nickname: String,
    status: String,
    email: String,
    login: String,
    password: String,
    preferred: bool,
    provider: String,
    created: DateTime<Utc>, // UTC timestamp for creation time
    updated: DateTime<Utc>, // UTC timestamp for update time
}


### request payload

The following data payload will be used to create a record in edgedb and used for retrieval purposes
```json
{
  "name": {
    "firstname": "tony",
    "lastname": "stark",
  },
  "nickname": "ironman",
  "status": "PENDING",
  "email": "ironman@avengers.com",
  "login": "@ironman",
  "password": "$2b$12$lH23QLU6pRf9g8jJW91HvOrPbGDFzwZ6x8Lz0jKkMQ7Bmgf1Sw9He",
  "preferred": true,
  "provider": "ardency"
}
```

In [12]:
#[tokio::main]
async fn main() -> Result<()> {
    let db = Surreal::new::<Ws>("localhost:8000").await?;

    // Signin as a namespace, database, or root user
    db.signin(Root {
        username: "root",
        password: "root",
    })
    .await?;

    // Select a specific namespace and database
    db.use_ns("labs").use_db("labs").await?;

    // get current time
    let current_time = Utc::now(); 

    // Create a new person with a random ID
    let tobie: Vec<Person> = db
        .create("person")
        .content(Person {
            id: None,
            nickname: "ironman".into(),
            name: Name {
                firstname: "Tony".into(),
                lastname: "Stark".into(),
            },
            identity: Identity {
                login: "@ironman".into(),
                email: "ironman@avengers.com".into(),
                password: "$2b$12$lH23QLU6pRf9g8jJW91HvOrPbGDFzwZ6x8Lz0jKkMQ7Bmgf1Sw9He".into(),
                preferred: true,
                providers: vec!(Provider {
                    name: "ardency".into(),
                })
            },
            
            created: current_time,
            updated: current_time, 
        })
        .await?;

    // Select all people records
    let people: Vec<Person> = db.select("person").await?;

    println!("{:?}",people);

    Ok(())
}

main()

Err(Failed to convert `[{ id: person:⟨018b6377-0003-7307-a07d-c351272be86b⟩ }, { id: person:01HDHQD6X9NKMTSSM0NZKK47WR }, { created: '2023-10-24T21:33:41.196297Z', id: person:d4y9zm0y9l5kfwqeiwnb, identity: { email: 'ironman@avengers.com', login: '@ironman', password: '$2b$12$lH23QLU6pRf9g8jJW91HvOrPbGDFzwZ6x8Lz0jKkMQ7Bmgf1Sw9He', preferred: true, providers: [{ name: 'ardency' }] }, name: { firstname: 'Tony', lastname: 'Stark' }, nickname: 'ironman', updated: '2023-10-24T21:33:41.196297Z' }]` to `T`: missing field `nickname`

Caused by:
    Failed to convert `[{ id: person:⟨018b6377-0003-7307-a07d-c351272be86b⟩ }, { id: person:01HDHQD6X9NKMTSSM0NZKK47WR }, { created: '2023-10-24T21:33:41.196297Z', id: person:d4y9zm0y9l5kfwqeiwnb, identity: { email: 'ironman@avengers.com', login: '@ironman', password: '$2b$12$lH23QLU6pRf9g8jJW91HvOrPbGDFzwZ6x8Lz0jKkMQ7Bmgf1Sw9He', preferred: true, providers: [{ name: 'ardency' }] }, name: { firstname: 'Tony', lastname: 'Stark' }, nickname: 'ironman', up